# Bowpi CCH - First Credit - Sagemaker POC - Linear Tunning


### Labeled Data

DirectSale request for features and Cubo Cortes from Data Warehouse for target variable

### Binary Classification

Since we have true labels to aim for, we'll take a **supervised learning** approach and train a binary classifier to sort data into one of our two transaction classes: good or bad client.  We'll train a model on training data and see how well it generalizes on some test data.

The notebook will be broken down into a few steps:
* Loading and exploring the data
* Splitting the data into train/test sets
* Defining and training a LinearLearner, binary classifier
* Making improvements on the model
* Evaluating and comparing model test performance

### Making Improvements

A lot of this notebook will focus on making improvements, as discussed in [this SageMaker blog post](https://aws.amazon.com/blogs/machine-learning/train-faster-more-flexible-models-with-amazon-sagemaker-linear-learner/). Specifically, we'll address techniques for:

1. **Tuning a model's hyperparameters** and aiming for a specific metric, such as high recall or precision.
2. **Managing class imbalance**, which is when we have many more training examples in one class than another (in this case, many more valid transactions than bad).

---

First, import the usual resources.

In [1]:
import io
import os
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 

import boto3
import sagemaker
from sagemaker import get_execution_role

%matplotlib inline

I'm storing my **SageMaker variables** in the next cell:
* sagemaker_session: The SageMaker session we'll use for training models.
* bucket: The name of the default S3 bucket that we'll use for data storage.
* role: The IAM role that defines our data and model permissions.

In [2]:
# sagemaker session, role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# S3 bucket name
bucket = sagemaker_session.default_bucket()


## Loading and Exploring the Data

Next, I am loading the data from S3 `xtest.csv` and `xtrain.csv`. 

It's important to look at the distribution of data since this will inform how we develop a a good client model. We'll want to know: How many data points we have to work with, the number and type of features, and finally, the distribution of data over the classes (good or bad).

In [3]:
# only have to run once
#!aws s3 cp s3://XX/datascience/dataset/20200415/directSalecClean/xtest.csv xtest.csv
#!aws s3 cp s3://XX/datascience/dataset/20200415/directSalecClean/xtrain.csv xtrain.csv

In [4]:
# read in the csv file
directSaleTrain = pd.read_csv('xtrain.csv')
directSaleTest = pd.read_csv('xtest.csv')
directSale = pd.concat([directSaleTrain, directSaleTest], axis=0).reset_index()

# print out some data
print('Data shape (rows, cols): ', directSale.shape)
print()
directSale.head()

Data shape (rows, cols):  (17719, 191)



,index,maxmora,cl_unq_act_act_fnacimiento,cl_unq_act_act_finicionegocio,cl_unq_act_act_fechasolicitud,numero_cuotas,cl_unq_act_act_personasdependientes,cl_unq_act_act_plazosolicitado,cl_unq_act_act_diasolicitado,cl_unq_act_act_tiempoentrecompras,...,cl_unq_act_act_estresfamiliacostos_na,cl_unq_act_act_negocioaproxegresos_na,cl_unq_act_act_fechasolicitud_month,cl_unq_act_act_totalgastosfam_ratio,cl_unq_act_act_negociototalingresos_ratio,cl_unq_act_act_totalbienes_ratio,cl_unq_act_act_totalgastosfamvar_log,cl_unq_act_act_negociototalingresosvar_log,cl_unq_act_act_totalbienesvar_log,cl_unq_act_act_montosolicitadovar_log
0,0,0,23,17/03/2011,06/03/2020,0.052632,0.06,0.37931,0.366667,0.000429,...,0.0,0.0,0.454545,0.029331,0.073073,0.005926,0.738372,0.600858,0.584507,0.545732
1,1,1,43,13/10/2016,18/08/2019,0.052632,0.08,0.37931,0.800000,0.000229,...,0.0,0.0,0.636364,0.013749,0.015682,0.000206,0.653488,0.387697,0.289613,0.545732
2,2,1,30,04/06/2015,15/08/2019,0.000000,0.06,0.37931,0.466667,0.000429,...,0.0,0.0,0.636364,0.029331,0.052386,0.005811,0.693023,0.496424,0.546655,0.423781
3,3,0,25,01/02/2014,11/03/2020,0.052632,0.08,0.37931,0.800000,0.000229,...,0.0,0.0,0.909091,0.013749,0.030364,0.002386,0.715116,0.560801,0.554577,0.722561
4,4,0,22,02/09/2012,06/07/2020,0.263158,0.10,0.37931,0.033333,0.000200,...,0.0,0.0,0.454545,0.035747,0.049049,0.004720,0.713953,0.486409,0.528169,0.423781


In [5]:
TARGET = ['maxmora']
date_var = ['cl_unq_act_act_finicionegocio', 'cl_unq_act_act_fechasolicitud']
drop_var = ['index']
FEATURES = [feat for feat in directSaleTrain.columns.to_list() if feat not in TARGET + date_var + drop_var]

data = directSale.copy().drop(columns=date_var)

### Calculate the percentage of bad clients data

Take a look at the distribution of this client data over the classes, good and bad. 

Count up the number of data points in each class and calculate the *percentage* of the data points that are bad.

In [6]:
# Calculate the fraction of data points that are bad
def target_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; bad.
       :param transaction_df: Dataframe of all transaction data points; has a column 'maxmora'
       :return: A fractional percentage of target data points/all points
    '''
    # counts for all classes
    counts = transaction_df[TARGET[0]].value_counts()
    
    # get fraudulent and valid cnts
    target_cnts = counts[1]
    valid_cnts = counts[0]
    
    # calculate percentage of fraudulent data
    target_percentage = target_cnts/(target_cnts+valid_cnts)
    
    return target_percentage


Test out your code by calling your function and printing the result.

In [7]:
# call the function to calculate the fraud percentage
data_percentage = target_percentage(data)

print('Bad Clients percentage = ', data_percentage)
print('Total # of bad clients pts: ', data_percentage*data.shape[0])
print('Out of (total) pts: ', data.shape[0])


Bad Clients percentage =  0.252666629042271
Total # of bad clients pts:  4477.0
Out of (total) pts:  17719


### Split into train/test datasets

In this example, we'll want to evaluate the performance of a target classifier; training it on some training data and testing it on *test data* that it did not see during the training process. So, we'll need to split the data into separate training and test sets.

This function should:
* Shuffle the data, randomly
* Split it into two sets according to the parameter `train_frac`
* Get train/test features and labels
* Return the tuples: (train_features, train_labels), (test_features, test_labels)

In [8]:
# split into train/test
def train_test_split(transaction_df, train_frac= 0.7, seed=1):
    '''Shuffle the data and randomly split into train and test sets;
       separate the class labels (the column in transaction_df) from the features.
       :param df: Dataframe of all client credit data
       :param train_frac: The decimal fraction of data that should be training data
       :param seed: Random seed for shuffling and reproducibility, default = 1
       :return: Two tuples (in order): (train_features, train_labels), (test_features, test_labels)
       '''
    
    # convert the df into a matrix for ease of splitting
    df_matrix = transaction_df.values
    
    # shuffle the data
    np.random.seed(seed)
    np.random.shuffle(df_matrix)
    
    # split the data
    train_size = int(df_matrix.shape[0] * train_frac)
    # features are all but last column
    train_features  = df_matrix[:train_size, 1:]
    # class labels *are* last column
    train_labels = df_matrix[:train_size, 0]
    # test data
    test_features = df_matrix[train_size:, 1:]
    test_labels = df_matrix[train_size:, 0]
    
    return (train_features, train_labels), (test_features, test_labels)


### Test Cell

In the cells below, I'm creating the train/test data and checking to see that result makes sense. The tests below test that the above function splits the data into the expected number of points and that the labels are indeed, class labels (0, 1).

In [9]:
# get train/test data
(train_features, train_labels), (test_features, test_labels) = train_test_split(data, train_frac=0.7)


In [10]:
# manual test

# for a split of 0.7:0.3 there should be ~2.33x as many training as test pts
print('Training data pts: ', len(train_features))
print('Test data pts: ', len(test_features))
print()

# take a look at first item and see that it aligns with first row of data
print('First item: \n', train_features[0])
print('Label: ', train_labels[0])
print()

# test split
assert len(train_features) > 2.333*len(test_features), \
        'Unexpected number of train/test points for a train_frac=0.7'
# test labels
assert np.all(train_labels)== 0 or np.all(train_labels)== 1, \
        'Train labels should be 0s or 1s.'
assert np.all(test_labels)== 0 or np.all(test_labels)== 1, \
        'Test labels should be 0s or 1s.'
print('Tests passed!')

Training data pts:  12403
Test data pts:  5316

First item: 
 [0.00000000e+00 4.20000000e+01 0.00000000e+00 2.00000000e-01
 5.86206897e-01 0.00000000e+00 5.71428571e-04 6.66666667e-03
 3.33333333e-02 0.00000000e+00 1.51818789e-02 4.57142857e-03
 4.02452254e-02 2.27821786e-02 2.35618173e-02 1.07274556e-02
 0.00000000e+00 3.83333333e-02 1.06912464e-01 4.22386008e-02
 8.57142857e-03 9.16666667e-03 6.54761905e-03 4.90903272e-02
 0.00000000e+00 1.00000000e-02 8.42047331e-02 0.00000000e+00
 2.97029703e-02 1.11627907e-02 1.18251689e-02 2.25000000e-03
 3.63490386e-01 1.10630229e-01 2.70360078e-03 1.06912453e-01
 1.66666667e-03 4.83523252e-02 4.80546465e-01 2.39628249e-02
 3.66013072e-02 1.69129721e-01 9.44444444e-03 1.86773905e-01
 4.84848485e-03 6.08212445e-04 6.31578947e-02 3.33333333e-02
 1.07687390e-01 1.12185682e-02 5.03997219e-03 3.60000000e-03
 2.38095238e-03 4.44444444e-02 6.50000000e-03 8.43727073e-02
 1.00000000e-02 1.10000000e+01 2.00000000e+00 0.00000000e+00
 0.00000000e+00 5.00000

---
# Modeling

Now that we've uploaded your training data, it's time to define and train a model!

In this notebook, you'll define and train the SageMaker, built-in algorithm, [LinearLearner](https://sagemaker.readthedocs.io/en/stable/linear_learner.html). 

A LinearLearner has two main applications:
1. For regression tasks in which a linear line is fit to some data points, and you want to produce a predicted output value given some data point (example: predicting house prices given square area).
2. For binary classification, in which a line is separating two classes of data and effectively outputs labels; either 1 for data that falls above the line or 0 for points that fall on or below the line.

<img src='notebook_ims/linear_separator.png' width=40% />

In this case, we'll be using it for case 2, and we'll train it to separate data into our two classes: good or bad. 

### Create a LinearLearner Estimator

All estimators require some constructor arguments to be passed in. For LinearLearner estimator, we use the documentation [LinearLearner documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html) as a resource. This takes in a lot of arguments, but not all are required. Later, we will discuss some specific hyperparameters and their use cases.

#### Instance Types

It is suggested that you use instances that are available in the free tier of usage: `'ml.c4.xlarge'` for training and `'ml.t2.medium'` for deployment.

In [37]:
# import LinearLearner
from sagemaker import LinearLearner

# specify an output path
prefix = 'firstCreditLinearTunning'
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate LinearLearner
linear = LinearLearner(role=role,
                       train_instance_count=1, 
                       train_instance_type='ml.c4.xlarge',
                       predictor_type='binary_classifier',
                       output_path=output_path,
                       sagemaker_session=sagemaker_session,
                       epochs=15)


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Convert data into a RecordSet format

Next, prepare the data for a built-in model by converting the train features and labels into numpy array's of float values. Then we can use the [record_set function](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner.record_set) to format the data as a RecordSet and prepare it for training!

In [38]:
# convert features/labels to numpy
train_x_np = train_features.astype('float32')
train_y_np = train_labels.astype('float32')

# create RecordSet
formatted_train_data = linear.record_set(train_x_np, labels=train_y_np)

### Train the Estimator

After instantiating your estimator, train it with a call to `.fit()`, passing in the formatted training data.

In [45]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-04-15 20:29:19 Starting - Starting the training job...
2021-04-15 20:29:43 Starting - Launching requested ML instancesProfilerReport-1618518559: InProgress
.........
2021-04-15 20:31:03 Starting - Preparing the instances for training.........
2021-04-15 20:32:46 Downloading - Downloading input data...
2021-04-15 20:33:18 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/15/2021 20:33:24 INFO 139675246659392] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'ini

[2021-04-15 20:33:26.892] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 9, "duration": 519, "num_examples": 13, "num_bytes": 9872788}
#metrics {"StartTime": 1618518806.8924556, "EndTime": 1618518806.8925538, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 0}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.5077975947062174, "count": 1, "min": 0.5077975947062174, "max": 0.5077975947062174}}}

#metrics {"StartTime": 1618518806.8926592, "EndTime": 1618518806.8926811, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 3, "model": 1}, "Metrics": {"train_binary_classification_cross_entropy_objective": {"sum": 0.5082383448282878, "count": 1, "min": 0.5082383448282878, "max": 0.5082383448282878}}}

#metrics {"StartTime": 1618518806.892738, "EndTime": 1618518806.8927557, "Dimensions": {"Algorithm": "Linear Learner", "Hos


2021-04-15 20:33:44 Uploading - Uploading generated training model
2021-04-15 20:33:44 Completed - Training job completed
Training seconds: 56
Billable seconds: 56
CPU times: user 655 ms, sys: 66 ms, total: 721 ms
Wall time: 4min 42s


### Deploy the trained model

Deploy your model to create a predictor. We'll use this to make predictions on our test data and evaluate the model.

In [46]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


---------------------!CPU times: user 361 ms, sys: 16.9 ms, total: 378 ms
Wall time: 10min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to the test data.

According to the deployed [predictor documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearnerPredictor), this predictor expects an `ndarray` of input features and returns a list of Records.
> "The prediction is stored in the "predicted_label" key of the `Record.label` field."

Let's first test our model on just one test point, to see the resulting list.

In [49]:
# test one prediction
test_x_np = test_features.astype('float32')
result = linear_predictor.predict(test_x_np[0])

print(result)

[label {
  key: "predicted_label"
  value {
    float32_tensor {
      values: 0.0
    }
  }
}
label {
  key: "score"
  value {
    float32_tensor {
      values: 0.08429912477731705
    }
  }
}
]


### Helper function for evaluation


The provided function below, takes in a deployed predictor, some test features and labels, and returns a dictionary of metrics; calculating false negatives and positives as well as recall, precision, and accuracy.

In [52]:
# code to evaluate the endpoint on test data
# returns a variety of model metrics
def evaluate(predictor, test_features, test_labels, verbose=True):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return binary classification metrics.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Class labels for test data
    :param verbose: If True, prints a table of all performance metrics
    :return: A dictionary of performance metrics.
    """
    
    # We have a lot of test data, so we'll split it into batches of 100
    # split the test data set into batches and evaluate using prediction endpoint    
    prediction_batches = [predictor.predict(batch) for batch in np.array_split(test_features, 100)]
    
    # LinearLearner produces a `predicted_label` for each data point in a batch
    # get the 'predicted_label' for every point in a batch
    test_preds = np.concatenate([np.array([x.label['predicted_label'].float32_tensor.values[0] for x in batch]) 
                                 for batch in prediction_batches])
    
    # calculate true positives, false positives, true negatives, false negatives
    tp = np.logical_and(test_labels, test_preds).sum()
    fp = np.logical_and(1-test_labels, test_preds).sum()
    tn = np.logical_and(1-test_labels, 1-test_preds).sum()
    fn = np.logical_and(test_labels, 1-test_preds).sum()
    
    # calculate binary classification metrics
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + fp + tn + fn)
    
    # printing a table of metrics
    if verbose:
        print(pd.crosstab(test_labels, test_preds, rownames=['actual (row)'], colnames=['prediction (col)']))
        print("\n{:<11} {:.3f}".format('Recall:', recall))
        print("{:<11} {:.3f}".format('Precision:', precision))
        print("{:<11} {:.3f}".format('Accuracy:', accuracy))
        print("{:<11} {:.3f}".format('TPR:', tpr))
        print("{:<11} {:.3f}".format('TNR:', tnr))
        print()
        
    return {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn, 
            'Precision': precision, 'Recall': recall, 'Accuracy': accuracy,
            'TPR': tpr, 'TNR': tnr}


### Test Results

The cell below runs the `evaluate` function. 

The code assumes that you have a defined `predictor` and `test_features` and `test_labels` from previously-run cells.

In [53]:
print('Metrics for simple, LinearLearner.\n')

# get metrics for linear predictor
metrics = evaluate(linear_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True) # verbose means we'll print out the metrics


Metrics for simple, LinearLearner.

prediction (col)   0.0  1.0
actual (row)               
0.0               3743  170
1.0               1180  223

Recall:     0.159
Precision:  0.567
Accuracy:   0.746
TPR:        0.159
TNR:        0.957



We can see that this model gets a fair accuracy of 74.6% ! But it still misclassifies our bad cases, which results in much lower values for recall and precision.

Next, let's delete this endpoint and discuss ways to improve this model.

## Delete the Endpoint

I've added a convenience function to delete prediction endpoints after we're done with them. And if you're done evaluating the model, you should delete your model endpoint!

In [54]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))

In [55]:
# delete the predictor endpoint 
delete_endpoint(linear_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-04-15-20-34-01-833


---

# Model Improvements

The default LinearLearner got a high accuracy, but still classified bad and valid data points incorrectly. Specifically classifying more than 170 points as false negatives (incorrectly labeled, bad clients), and a little over 1180 points as false positives (incorrectly labeled, valid transactions). Let's think about what, during training, could cause this behavior and what we could improve.

**1. Model optimization**
* To train according to specific product demands and goals, we do not want to optimize for accuracy only. Instead, we want to optimize for a metric that can help us decrease the number of false positives or negatives. 

<img src='notebook_ims/precision_recall.png' width=40% />
     
In this notebook, we'll look at different cases for tuning a model and make an optimization decision, accordingly.

**2. Imbalanced training data**
* At the start of this notebook, we saw that only about 0.25% of the data was labeled as bad client's credit. So, even if a model labels **all** of our data as valid, it will still have a high accuracy. 
* This may result in some overfitting towards valid data, which accounts for some **false negatives**; cases in which bad clients data (1) is incorrectly characterized as valid (0).

So, let's address these issues in order; first, tuning our model and optimizing for a specific metric during training, and second, accounting for class imbalance in the training set. 


## Improvement: Model Tuning

Optimizing according to a specific metric is called **model tuning**, and SageMaker provides a number of ways to automatically tune a model.


### Create a LinearLearner and tune for higher precision 

**Scenario:**
* A bank has asked you to build a model that detects cases of fraud with an accuracy of about 65%. 

In this case, we want to build a model that has as many true positives and as few false negatives, as possible. This corresponds to a model with a high **recall**: true positives / (true positives + false negatives). 

To aim for a specific metric, LinearLearner offers the hyperparameter `binary_classifier_model_selection_criteria`, which is the model evaluation criteria for the training dataset. A reference to this parameter is in [LinearLearner's documentation](https://sagemaker.readthedocs.io/en/stable/linear_learner.html#sagemaker.LinearLearner). We'll also have to further specify the exact value we want to aim for; read more about the details of the parameters, [here](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html).

I will assume that performance on a training set will be within about 5% of the performance on a test set. So, for a recall of about 60%, I'll aim for a bit higher, 65%.

In [72]:
# instantiate a LinearLearner
# tune the model for a higher recall
linear_recall = LinearLearner(role=role,
                              train_instance_count=1, 
                              train_instance_type='ml.c4.xlarge',
                              predictor_type='binary_classifier',
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              epochs=15,
                              binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                              target_recall=0.6) # 90% recall


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Train the tuned estimator

Fit the new, tuned estimator on the formatted training data.

In [73]:
%%time 
# train the estimator on formatted training data
linear_recall.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-04-15 22:10:19 Starting - Starting the training job...
2021-04-15 22:10:42 Starting - Launching requested ML instancesProfilerReport-1618524619: InProgress
......
2021-04-15 22:11:42 Starting - Preparing the instances for training.........
2021-04-15 22:13:03 Downloading - Downloading input data...
2021-04-15 22:13:43 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script

2021-04-15 22:14:15 Uploading - Uploading generated training model[04/15/2021 22:14:05 INFO 140055115528000] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_sca


2021-04-15 22:14:43 Completed - Training job completed
ProfilerReport-1618524619: NoIssuesFound
Training seconds: 84
Billable seconds: 84
CPU times: user 680 ms, sys: 54.3 ms, total: 735 ms
Wall time: 4min 42s


### Deploy and evaluate the tuned estimator

Deploy the tuned predictor and evaluate it.

We hypothesized that a tuned model, optimized for a higher recall, would have fewer false negatives (bad clients  incorrectly labeled as valid); did the number of false negatives get reduced after tuning the model?

In [74]:
%%time 
# deploy and create a predictor
recall_predictor = linear_recall.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


---------------------!CPU times: user 343 ms, sys: 28.7 ms, total: 371 ms
Wall time: 10min 34s


In [75]:
print('Metrics for tuned (recall), LinearLearner.\n')

# get metrics for tuned predictor
metrics = evaluate(recall_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (recall), LinearLearner.

prediction (col)   0.0   1.0
actual (row)                
0.0               2780  1133
1.0                577   826

Recall:     0.589
Precision:  0.422
Accuracy:   0.678
TPR:        0.589
TNR:        0.710



## Delete the endpoint 

As always, when you're done evaluating a model, you should delete the endpoint. Below, I'm using the `delete_endpoint` helper function I defined earlier.

In [77]:
# delete the predictor endpoint 
delete_endpoint(recall_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-04-15-22-15-02-087


---
## Improvement: Managing Class Imbalance

We have a model that is tuned to get a higher recall, which aims to reduce the number of false negatives. Earlier, we discussed how class imbalance may actually bias our model towards predicting that all transactions are valid, resulting in higher false negatives and true negatives. It stands to reason that this model could be further improved if we account for this imbalance.

To account for class imbalance during training of a binary classifier, LinearLearner offers the hyperparameter, `positive_example_weight_mult`, which is the weight assigned to positive (1, bad clients) examples when training a binary classifier. The weight of negative examples (0, valid) is fixed at 1. 

### Create a LinearLearner with a `positive_example_weight_mult` parameter

In **addition** to tuning a model for higher recall (you may use `linear_recall` as a starting point), you should *add* a parameter that helps account for class imbalance. From the [hyperparameter documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ll_hyperparameters.html) on `positive_example_weight_mult`, it reads:
> "If you want the algorithm to choose a weight so that errors in classifying negative vs. positive examples have equal impact on training loss, specify `balanced`."

You could also put in a specific float value, in which case you'd want to weight positive examples more heavily than negative examples, since there are fewer of them.

In [66]:
# instantiate a LinearLearner

# include params for tuning for higher recall
# *and* account for class imbalance in training data
linear_balanced = LinearLearner(role=role,
                                train_instance_count=1, 
                                train_instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=20,
                                binary_classifier_model_selection_criteria='precision_at_target_recall', # target recall
                                target_recall=0.58,
                                positive_example_weight_mult='balanced')


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Train the balanced estimator

Fit the new, balanced estimator on the formatted training data.

In [67]:
%%time 
# train the estimator on formatted training data
linear_balanced.fit(formatted_train_data)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-04-15 21:24:23 Starting - Starting the training job...
2021-04-15 21:24:47 Starting - Launching requested ML instancesProfilerReport-1618521863: InProgress
.........
2021-04-15 21:26:07 Starting - Preparing the instances for training.........
2021-04-15 21:27:49 Downloading - Downloading input data
2021-04-15 21:27:49 Training - Downloading the training image...
2021-04-15 21:28:11 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/15/2021 21:28:15 INFO 140376322160448] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method'

[2021-04-15 21:28:21.929] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 23, "duration": 523, "num_examples": 13, "num_bytes": 9872788}
#metrics {"StartTime": 1618522101.9292324, "EndTime": 1618522101.9293232, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.9185304616292318, "count": 1, "min": 0.9185304616292318, "max": 0.9185304616292318}}}

#metrics {"StartTime": 1618522101.9294252, "EndTime": 1618522101.9294472, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.9191836700439453, "count": 1, "min": 0.9191836700439453, "max": 0.9191836700439453}}}

#metrics {"StartTime": 1618522101.9295092, "EndTime": 1618522101.9295266, "Dimensions": {"Algorithm": 


2021-04-15 21:28:48 Uploading - Uploading generated training model
2021-04-15 21:28:48 Completed - Training job completed
Training seconds: 52
Billable seconds: 52
CPU times: user 709 ms, sys: 48.8 ms, total: 758 ms
Wall time: 4min 42s


### Deploy and evaluate the balanced estimator

Deploy the balanced predictor and evaluate it. Do the results match with your expectations?

In [69]:
%%time 
# deploy and create a predictor
balanced_predictor = linear_balanced.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!CPU times: user 335 ms, sys: 18.8 ms, total: 354 ms
Wall time: 9min 32s


In [70]:
print('Metrics for balanced, LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(balanced_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for balanced, LinearLearner.

prediction (col)   0.0   1.0
actual (row)                
0.0               2856  1057
1.0                609   794

Recall:     0.566
Precision:  0.429
Accuracy:   0.687
TPR:        0.566
TNR:        0.730



## Delete the endpoint 

When you're done evaluating a model, you should delete the endpoint.

In [71]:
# delete the predictor endpoint 
delete_endpoint(balanced_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-04-15-21-35-26-744


A note on metric variability: 

The above model is tuned for the best possible precision with recall fixed at about 58%. The recall is fixed at 58% during training, but may vary when we apply our trained model to a test set of data.

---
## Model Design

Now that you've seen how to tune and balance a LinearLearner. Create, train and deploy your own model. This exercise is meant to be more open-ended, so that you get practice with the steps involved in designing a model and deploying it.

### Train and deploy a LinearLearner with appropriate hyperparameters, according to the given scenario

**Scenario:**
* A Bank has asked you to build a model that optimizes for a good user experience; users should only ever have up to about 30% of their valid clients flagged as bad.

This requires that you make a design decision: Given the above scenario, what metric (and value) should you aim for during training?

You may assume that performance on a training set will be within about 5-10% of the performance on a test set. For example, if you get 80% on a training set, you can assume that you'll get between about 70-90% accuracy on a test set.

Your final model should account for class imbalance and be appropriately tuned. 

In [62]:
%%time
# instantiate and train a LinearLearner

# include params for tuning for higher precision
# *and* account for class imbalance in training data
linear_precision = LinearLearner(role=role,
                                train_instance_count=1, 
                                train_instance_type='ml.c4.xlarge',
                                predictor_type='binary_classifier',
                                output_path=output_path,
                                sagemaker_session=sagemaker_session,
                                epochs=15,
                                binary_classifier_model_selection_criteria='recall_at_target_precision',
                                target_precision=0.6,
                                positive_example_weight_mult='balanced')


# train the estimator on formatted training data
linear_precision.fit(formatted_train_data)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


2021-04-15 21:07:47 Starting - Starting the training job...
2021-04-15 21:08:17 Starting - Launching requested ML instancesProfilerReport-1618520866: InProgress
......
2021-04-15 21:09:17 Starting - Preparing the instances for training............
2021-04-15 21:11:18 Downloading - Downloading input data...
2021-04-15 21:11:39 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/15/2021 21:11:44 INFO 140692695308096] Reading default configuration from /opt/amazon/lib/python3.7/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init

[2021-04-15 21:11:46.644] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 5, "duration": 489, "num_examples": 13, "num_bytes": 9872788}
#metrics {"StartTime": 1618521106.6444776, "EndTime": 1618521106.6447005, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 0}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.9665324401855468, "count": 1, "min": 0.9665324401855468, "max": 0.9665324401855468}}}

#metrics {"StartTime": 1618521106.6447856, "EndTime": 1618521106.6448023, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 1, "model": 1}, "Metrics": {"train_binary_classification_weighted_cross_entropy_objective": {"sum": 0.9785897572835286, "count": 1, "min": 0.9785897572835286, "max": 0.9785897572835286}}}

#metrics {"StartTime": 1618521106.6448433, "EndTime": 1618521106.644854, "Dimensions": {"Algorithm": "Lin


2021-04-15 21:12:18 Uploading - Uploading generated training model
2021-04-15 21:12:18 Completed - Training job completed
Training seconds: 56
Billable seconds: 56
CPU times: user 698 ms, sys: 42.9 ms, total: 741 ms
Wall time: 4min 43s


This model trains for a fixed precision of 60%, and, under that constraint, tries to get as high a recall as possible.

In [63]:
%%time 
# deploy and evaluate a predictor
precision_predictor = linear_precision.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


-------------------!CPU times: user 335 ms, sys: 9.01 ms, total: 344 ms
Wall time: 9min 32s


In [64]:
print('Metrics for tuned (precision), LinearLearner.\n')

# get metrics for balanced predictor
metrics = evaluate(precision_predictor, 
                   test_features.astype('float32'), 
                   test_labels, 
                   verbose=True)

Metrics for tuned (precision), LinearLearner.

prediction (col)   0.0  1.0
actual (row)               
0.0               3830   83
1.0               1285  118

Recall:     0.084
Precision:  0.587
Accuracy:   0.743
TPR:        0.084
TNR:        0.979



In [65]:
## IMPORTANT
# delete the predictor endpoint 
delete_endpoint(precision_predictor)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Deleted linear-learner-2021-04-15-21-12-29-917


## Final Cleanup!

* Double check that you have deleted all your endpoints.
* I'd also suggest manually deleting your S3 bucket, models, and endpoint configurations directly from your AWS console.

You can find thorough cleanup instructions, [in the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html).

---
# Conclusion

In this notebook, you saw how to train and deploy a LinearLearner in SageMaker. This model is well-suited for a binary classification task that involves specific design decisions and managing class imbalance in the training set.
